<a href="https://colab.research.google.com/github/claudelepere/ML_GitHub/blob/main/jobs_EN_11_0_12000_macro_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q accelerate
!pip install -q huggingface_hub
!pip install -q scikit-learn
!pip install -q transformers datasets  # transformers and datasets are Hugging Face libraries
!pip install -q wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 50.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import datetime
import json
import numpy as np
import os
import pandas as pd
import pickle
import sys
import time
import torch
import torch.nn.functional as F
import wandb

from datasets               import DatasetDict
from google.colab           import auth, drive, files, userdata
from huggingface_hub        import create_branch, create_repo, HfApi, login, upload_file, hf_hub_download, whoami
from huggingface_hub.errors import RepositoryNotFoundError
from sklearn.metrics        import f1_score, precision_score, recall_score, roc_auc_score, average_precision_score, accuracy_score, hamming_loss, classification_report,  precision_recall_fscore_support

from torch.utils.data       import DataLoader
from tqdm.auto              import tqdm
from transformers           import AdamW, EvalPrediction, LongformerTokenizerFast, LongformerForSequenceClassification, Trainer, TrainingArguments
from torch.nn               import BCEWithLogitsLoss, Module

## Google Cloud authentication

In [3]:
auth.authenticate_user()  # user = c.lepere@ictjob.be

## Get skills and jobs

In [4]:
skills         = 11
all_rows_low   = 0
all_rows_high  = 12000 # 120 12000 24000
num_datapoints = all_rows_high - all_rows_low

datasetDict_zip_file_name = f"dataset_EN_{skills}_{all_rows_low}_{all_rows_high}.zip"
datasetDict_dir_name      = os.path.splitext(datasetDict_zip_file_name)[0]

print(f"datasetDict_zip_file_name: {datasetDict_zip_file_name}")
print(f"datasetDict_dir_name     : {datasetDict_dir_name}")
print()

datasetDict_zip_file_name: dataset_EN_11_0_12000.zip
datasetDict_dir_name     : dataset_EN_11_0_12000



## Tune thresholds?

In [5]:
threshold_tuning = True

## Upload to HF Hub?

In [6]:
upload_to_HF = False
repo_id      = ''
timestamp    = ''


## Hugging Face Hub (HF Hub) authenticate

In [7]:
if upload_to_HF is True:

  os.environ["HF_TOKEN"] = userdata.get("HF_TOKEN")    # Store the key in os.environ
  hf_token               = os.environ.get('HF_TOKEN')

  login(token=hf_token)

  # Check
  user = whoami(token=hf_token)
  assert user['name'] == 'claudelepere', f"{user['name']} is not claudelepere"
  print(f"user: {user}")

## repo_id, branch, model and dataset repos on HF Hub
**1 repo = 1 model and 1 tokenizer**

**branch = revision**

In [8]:
if upload_to_HF is True:

  if threshold_tuning:
    #repo_id   = 'claudelepere/jobs_EN_11_0_24000_tuned_thresholds'
    repo_id   = 'claudelepere/jobs_EN_11_0_1200_tuned_thresholds'
  else:
    #repo_id   = 'claudelepere/jobs_EN_11_0_24000'
    repo_id   = 'claudelepere/jobs_EN_11_0_1200'
  timestamp = f"{datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"

  model_repoUrl   = create_repo(repo_id=repo_id, repo_type="model",   private=True, exist_ok=True)
  dataset_repoUrl = create_repo(repo_id=repo_id, repo_type="dataset", private=True, exist_ok=True)

  #create_branch(repo_id=repo_id, repo_type="model",   branch=branch, exist_ok=True)
  #create_branch(repo_id=repo_id, repo_type="dataset", branch=branch, exist_ok=True)

  print(f"Model Repo Url: {model_repoUrl} created successfully as a private repo")
  print(f"Dataset Repo Url: {dataset_repoUrl} created successfully as a private repo")

## HF model card
Model card here => README.md on the HF Hub.

In [9]:
model_card = """
---
tags:
- "24000"
---
# Model
Model fine-tuned on higly-imbalanced multilabel classification.

## Model details
- Language: English
- Task: Multilabel classification
- Architecture: Longformer
- Pretrained model: [allenai/longformer-base-4096](https://huggingface.co/allenai/longformer-base-4096)
- Framework: Pytorch
- Version 1.0.0

## Training Data
- skills: 11
- 24000 job datapoints

## Fine-tuning parameters
- batch size: 8
- gradient accumulation: 4
- fp16 precision
- input tokens max length: 1024
- epochs: 5
- learning rate: 1e-5
- tuned thresholds
- attention window size: 1024
"""

## Save locally and upload model card to HF Hub

In [10]:
if upload_to_HF is True:

  with open("model_card.md", "w") as f:
    f.write(model_card)

  upload_file(
      path_or_fileobj = 'model_card.md',  # path to a file or a file-like object
      path_in_repo    = 'README.md',
      repo_id         = repo_id
  )

In [11]:
"""
# Check the Python version
print(sys.version)
print()

# Get the installed packages (you can see that conda is not installed (do not install it))
!pip list
print()

# Check system information
!cat /etc/os-release
!uname -m
print()

# Check the GPU details (only if the runtime type is T4 GPU)
#!nvidia-smi
#print()

# Check RAM
!free -h
print()

# Check disk space
!df -h
print()

# Get environment variables
for key, value in os.environ.items():
    print(f"{key}: {value}")
"""
!python -V

Python 3.11.11


In [12]:
print(f"currentdir: {os.getcwd()}")

currentdir: /content


In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"device: {device}")

device: cuda


## Out Of Memory (OOM)

### OOM: reduce batch size
      small sizes (1 to 32):            PROs: better generalization in some cases
                                        CONs: may produce noisier gradients
      large sizes (128, 256, or higer): PROs: gradients are smoother, leading to more stable training
                                        CONs: poorer generalization (overfitting) in some cases
      intermediate sizes (32, 64):      combines the benefits of small and large sizes

In [14]:
batch_size = 8

### OOM: enable gradient accumulation

* compensate for smaller batch sizes by accumulating gradients over several steps
* **effective batch size** = per-device batch size x gradient acumulation steps
* in each iteration, the model computes the gradients, these gradients are immediately used to update the model parameters

WARNING: gradient_accumulation_steps may not be None => comment it in TrainingArguments

In [15]:
gradient_accumulation_steps = 4

### OOM: use PYTORCH_CUDA_ALLOC_CONF to handle memory fragmentation

In [16]:
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

### OOM: check for and kill zombie processes

In [17]:
!ps aux | grep python
!kill -9 <PID>
if torch.cuda.is_available():
  !nvidia-smi
  print(torch.cuda.memory_summary())

root          85  2.0  0.0      0     0 ?        Z    17:37   0:13 [python3] <defunct>
root          86  0.0  0.0  63748 50020 ?        S    17:37   0:00 python3 /usr/local/bin/colab-file
root         135  0.6  0.1 780708 127164 ?       Sl   17:37   0:04 /usr/bin/python3 /usr/local/bin/j
root        2227 12.5  1.4 12134304 1302008 ?    Ssl  17:45   0:20 /usr/bin/python3 -m colab_kernel_
root        2280  0.4  0.0 545100 20768 ?        Sl   17:45   0:00 /usr/bin/python3 /usr/local/lib/p
root        3151  0.0  0.0   7376  3444 ?        S    17:48   0:00 /bin/bash -c ps aux | grep python
root        3153  0.0  0.0   6484  2280 ?        S    17:48   0:00 grep python
/bin/bash: -c: line 1: syntax error near unexpected token `newline'
/bin/bash: -c: line 1: `kill -9 <PID>'
Wed Feb 19 17:48:13 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----

### OOM: use fp16 (half precision) mixed precision training
reduces memory requirements by up to 50%

In [18]:
fp = 'fp16'

### OOM: limit the number of GPU workers:
* 0 (default) or 1
* in Colab dataloader_num_workers = 1

### OOM: reduce model size or input tokens
* LongformerTokenizer.from_pretrained('allenai/longformer-base/large-4096'): large/base: 435M/149M parameters
* max_length: 4096 max for Longformer
* a single word can be equal to several tokens; stop words are **NOT discarded**!
* word_text_length_counts_sorted:
      jobs count                 : 50000
      jobs count under  512 words: 44794  89.59%
      jobs count under  640 words: 47894  95.79%
      jobs count under  768 words: 49123  98.25%
      jobs count under  896 words: 49691  99.38%
      jobs count under 1024 words: 49917  99.83%
      jobs count under 2048 words: 50000 100.00%
      jobs count under 4096 words: 50000 100.00%

In [19]:
#max_length =  768    #      37 min    #
max_length = 1024    #      38 min    # GPU RAM: 12.2 / 40 GB
#max_length = 2048    # 1 hr 10 min    # GPU RAM: 21.4 / 40 GB
#max_length = 4096    # 2 hr 10 min    # GPU RAM: 39.5 / 40 GB => OutOfMemoryError

### OOM: free up GPU memory

In [20]:
torch.cuda.empty_cache()

### OOM: reduce the number of transformers layers

In [21]:
# hidden_layers = 6  # default:12

## epoch
* 1 epoch is a complete pass through the entire training dataset
* with n datapoints and batch size = b, n/b iterations to complete 1 epoch
* 1 iteration is a single update of the model's parameters

In [22]:
epochs = 5

## learning rate
* A common rule is to scale the learning rate proportionaly with the effective batch size
* **note: get_linear_schedule_with_warmup**

In [23]:
learning_rate = 1e-5  # 1e-5 x 32/8

## threshold
default: 0.5

In [24]:
threshold = 0.5

## attention window size

In [25]:
attention_window = 1024 #512

## Upload and unzip job dataset

In [26]:
def upload_unzip_dataset(file_name=datasetDict_zip_file_name):
  # Check if the file exists
  if not os.path.exists(file_name):
    print(f"'{file_name}' not found in /content. Uploading...")
    uploaded_files = files.upload()                              # Prompt file upload dialog
    if file_name not in uploaded_files:
      raise FileNotFoundError(f"'{file_name}' was not uploaded. Please try again.")
    print(f"'{file_name}' successfully uploaded to /content")
    uploaded_file_name = list(uploaded_files.keys())[0]          # Get the name of the uploaded file

    !unzip {uploaded_file_name}

    unzipped_dir_name = os.path.splitext(uploaded_file_name)[0]
    assert unzipped_dir_name==datasetDict_dir_name, "unzipped_dir_name != datasetDict_dir_name"
  else:
    print(f"'{file_name}' already exists in /content.")

upload_unzip_dataset(datasetDict_zip_file_name)

'dataset_EN_11_0_12000.zip' not found in /content. Uploading...


Saving dataset_EN_11_0_12000.zip to dataset_EN_11_0_12000.zip
'dataset_EN_11_0_12000.zip' successfully uploaded to /content
Archive:  dataset_EN_11_0_12000.zip
  inflating: dataset_EN_11_0_12000/dataset_dict.json  
  inflating: dataset_EN_11_0_12000/test/data-00000-of-00001.arrow  
  inflating: dataset_EN_11_0_12000/test/dataset_info.json  
  inflating: dataset_EN_11_0_12000/test/state.json  
  inflating: dataset_EN_11_0_12000/train/data-00000-of-00001.arrow  
  inflating: dataset_EN_11_0_12000/train/dataset_info.json  
  inflating: dataset_EN_11_0_12000/train/state.json  
  inflating: dataset_EN_11_0_12000/validation/data-00000-of-00001.arrow  
  inflating: dataset_EN_11_0_12000/validation/dataset_info.json  
  inflating: dataset_EN_11_0_12000/validation/state.json  


## W&B initialization (not used now)

In [27]:
run_name = f"EN_{skills}_{all_rows_low}_{all_rows_high}_ml{max_length}_ep{epochs}_lr{learning_rate}_th{threshold}_at{attention_window}_{fp}"

if 'gradient_accumulation_steps' not in globals():
  run_name = f"{run_name}_ba{batch_size}"
else:
  run_name = f"{run_name}_ba{batch_size}x{gradient_accumulation_steps}"

print(f"run_name: {run_name}")

run_name: EN_11_0_12000_ml1024_ep5_lr1e-05_th0.5_at1024_fp16_ba8x4


In [28]:
os.environ["WANDB_API_KEY"] = userdata.get("WANDB_API_KEY")        # Store the key in os.environ
wandb_api_key               = os.environ.get('WANDB_API_KEY')
wandb.login(key=wandb_api_key)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: claudelepere (claudelepere-c-cile-cy) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [29]:
try:
  wandb.init(
      project = "skill_classification",
      name    = run_name,
      entity  = "claudelepere-c-cile-cy",
      config  = {
          "learning_rate": learning_rate,
          "epochs"       : epochs,
          "batch_size"   : batch_size
      }
  )
except wandb.CommError as err:
  print(f"CommError: {err}")
except Exception as exc:
  print(f"Exception: {exc}")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


## Create datasetDict (HF DatasetDict) = 3 HF Dataset, train, validation and test

In [30]:
datasetDict = DatasetDict.load_from_disk(datasetDict_dir_name)

In [31]:
print(f"datasetDict: {type(datasetDict)} {datasetDict.shape}\n{datasetDict}")
print(f"datasetDict.keys(): {datasetDict.keys()}")
print(f"datasetDict['train']:      {type(datasetDict['train'])}      {datasetDict['train'].shape}")
print(f"datasetDict['validation']: {type(datasetDict['validation'])} {datasetDict['validation'].shape}")
print(f"datasetDict['test']:       {type(datasetDict['test'])}       {datasetDict['test'].shape}")

datasetDict: <class 'datasets.dataset_dict.DatasetDict'> {'train': (9600, 8), 'validation': (1200, 8), 'test': (1200, 8)}
DatasetDict({
    train: Dataset({
        features: ['id', 'text', '390', '135', '136', '137', '138', '139'],
        num_rows: 9600
    })
    validation: Dataset({
        features: ['id', 'text', '390', '135', '136', '137', '138', '139'],
        num_rows: 1200
    })
    test: Dataset({
        features: ['id', 'text', '390', '135', '136', '137', '138', '139'],
        num_rows: 1200
    })
})
datasetDict.keys(): dict_keys(['train', 'validation', 'test'])
datasetDict['train']:      <class 'datasets.arrow_dataset.Dataset'>      (9600, 8)
datasetDict['validation']: <class 'datasets.arrow_dataset.Dataset'> (1200, 8)
datasetDict['test']:       <class 'datasets.arrow_dataset.Dataset'>       (1200, 8)


In [32]:
example = datasetDict['train'][0]
print(f"datasetDict['train'][0]: {type(example)} {example.keys()}\n{example}")

datasetDict['train'][0]: <class 'dict'> dict_keys(['id', 'text', '390', '135', '136', '137', '138', '139'])
{'id': 153918, 'text': "Proximus SpearIT - Senior Web Security Engineer   F5, Security, Firewall, LTM, ASM   Proximus SpearIT   Job description Install, deploy, manage and operate security solutions based on Web Application Firewall systems Mandatory knowledge of F5 Technologies (LTM and ASM) Good Knowledge of Firewall, Proxy, Networking (Routing & Switching) systems and technologies Testing of new configuration, equipment and releases Reporting on network/security usage and performance Participate to project and deployment of new security architecture Deliver technical solutions and improve the level of protection of the system Develop the technical documentation regarding the operational procedures Be part in the security incident response process and participate to On-Duty staffing outside business hours. Job requirements Advanced skills in F5 (LTM and ASM) architectures, solu

## Create labels (list), id2label (dict) and label2id (dict).
* dataset 7_1000_125_125  ,  48 labels
* dataset 7_128_18_54     ,  42 labels
* dataset 8910_1087_68_204, 206 labels
* dataset 11_1000         ,   6 labels

In [33]:
labels = [label for label in datasetDict['train'].features.keys() if label not in ['id', 'text']]
labels.sort()
print(f"labels: {type(labels)} {len(labels)}\n{labels}")

num_labels = len(labels)

id2label = {idx: label for idx, label in enumerate(labels)}
print(f"id2label: {type(id2label)} {len(id2label)}\n{id2label}")

label2id = {label: idx for idx, label in enumerate(labels)}
print(f"label2id: {type(label2id)} {len(label2id)}\n{label2id}")

labels: <class 'list'> 6
['135', '136', '137', '138', '139', '390']
id2label: <class 'dict'> 6
{0: '135', 1: '136', 2: '137', 3: '138', 4: '139', 5: '390'}
label2id: <class 'dict'> 6
{'135': 0, '136': 1, '137': 2, '138': 3, '139': 4, '390': 5}


## Load the pretrained tokenizer and the model

In [34]:
model_name = "allenai/longformer-base-4096"

In [35]:
tokenizer = LongformerTokenizerFast.from_pretrained(model_name)

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

In [36]:
model = LongformerForSequenceClassification.from_pretrained(
    model_name,
    num_labels   = num_labels,
    id2label     = id2label,
    label2id     = label2id,
    problem_type = 'multi_label_classification'
)

# Configure attention window size
model.config.attention_window = attention_window

pytorch_model.bin:   0%|          | 0.00/597M [00:00<?, ?B/s]

Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [37]:
optimizer = AdamW(model.parameters(), lr=learning_rate)

/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


## Tokenize ('input_ids' and 'attention_mask'), add 'global_attention_mask' (for Longformer), add 'labels'

In [38]:
def preprocess_data(examples, indices):
  # Step 1: Extract text and tokenize
  text = examples['text']             # Batch of texts
  encoding = tokenizer(
      text,                           # Tokenize text
      truncation     = True,
      padding        = 'max_length',
      max_length     = max_length,
      return_tensors = 'pt'           # Return PyTorch tensors
  )

  # Step 2: Create and add the global attention mask
  global_attention_mask             = torch.zeros_like(encoding['input_ids'])  # Initialize global attention mask with zeros (same shape as input_ids)
  global_attention_mask[:, 0]       = 1                                        # Set global attention on the first token ([CLS], token ID=0) in each sequence
  encoding['global_attention_mask'] = global_attention_mask                    # Add the global_attention_mask to the batch

  # Step 3: Create and populate the label matrix
  labels_matrix = torch.zeros((len(text), len(labels)), dtype=torch.float32)   # Create an empty label matrix
  #print(f"labels_matrix: {type(labels_matrix)} {labels_matrix.shape}")
  #---------Populate label matrix
  for idx, label in enumerate(labels):
    #print(f"idx:{idx} label:{label}")
    if label in examples:
      labels_matrix[:, idx] = torch.tensor(
          [1.0 if val else 0.0 for val in examples[label]],
          dtype=torch.float32
          )
  print(f"labels_matrix: {type(labels_matrix)} {labels_matrix.shape}")

  encoding['labels'] = labels_matrix                                           # Add labels to the encoding
  print(f"encoding['labels']: {type(encoding['labels'])} {encoding['labels'].shape}")

  # encoding: <class 'transformers.tokenization_utils_base.BatchEncoding'> dict_keys(['input_ids', 'attention_mask', 'global_attention_mask', 'labels'])
  #   'input_ids': tensor([[
  #   'attention_mask': tensor([[
  #   'global_attention_mask': tensor([[
  #   'labels': tensor([[
  #print(f"1 preprocess_data call: encoding: {type(encoding)} {encoding.keys()}")

  return encoding

## Create encoded_dataset (datasets.dataset_dict.DatasetDict) = 3 encoded datasets.arrow_dataset.Dataset, train, validation and test

In [39]:
encoded_dataset = datasetDict.map(
    preprocess_data,
    batched        = True,
    remove_columns = datasetDict['train'].column_names,
    with_indices   = True
)

print(f"Zencoded_dataset: {type(encoded_dataset)} shape={encoded_dataset.shape}")

Map:   0%|          | 0/9600 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/597M [00:00<?, ?B/s]

labels_matrix: <class 'torch.Tensor'> torch.Size([1000, 6])
encoding['labels']: <class 'torch.Tensor'> torch.Size([1000, 6])
labels_matrix: <class 'torch.Tensor'> torch.Size([1000, 6])
encoding['labels']: <class 'torch.Tensor'> torch.Size([1000, 6])
labels_matrix: <class 'torch.Tensor'> torch.Size([1000, 6])
encoding['labels']: <class 'torch.Tensor'> torch.Size([1000, 6])
labels_matrix: <class 'torch.Tensor'> torch.Size([1000, 6])
encoding['labels']: <class 'torch.Tensor'> torch.Size([1000, 6])
labels_matrix: <class 'torch.Tensor'> torch.Size([1000, 6])
encoding['labels']: <class 'torch.Tensor'> torch.Size([1000, 6])
labels_matrix: <class 'torch.Tensor'> torch.Size([1000, 6])
encoding['labels']: <class 'torch.Tensor'> torch.Size([1000, 6])
labels_matrix: <class 'torch.Tensor'> torch.Size([1000, 6])
encoding['labels']: <class 'torch.Tensor'> torch.Size([1000, 6])
labels_matrix: <class 'torch.Tensor'> torch.Size([1000, 6])
encoding['labels']: <class 'torch.Tensor'> torch.Size([1000, 6])


Map:   0%|          | 0/1200 [00:00<?, ? examples/s]

labels_matrix: <class 'torch.Tensor'> torch.Size([1000, 6])
encoding['labels']: <class 'torch.Tensor'> torch.Size([1000, 6])
labels_matrix: <class 'torch.Tensor'> torch.Size([200, 6])
encoding['labels']: <class 'torch.Tensor'> torch.Size([200, 6])


Map:   0%|          | 0/1200 [00:00<?, ? examples/s]

labels_matrix: <class 'torch.Tensor'> torch.Size([1000, 6])
encoding['labels']: <class 'torch.Tensor'> torch.Size([1000, 6])
labels_matrix: <class 'torch.Tensor'> torch.Size([200, 6])
encoding['labels']: <class 'torch.Tensor'> torch.Size([200, 6])
Zencoded_dataset: <class 'datasets.dataset_dict.DatasetDict'> shape={'train': (9600, 4), 'validation': (1200, 4), 'test': (1200, 4)}


In [40]:
encoded_dataset.set_format('torch')
train_dataset      = encoded_dataset['train']
validation_dataset = encoded_dataset['validation']
test_dataset       = encoded_dataset['test']

print(f"train_dataset_tensor:                          {type(train_dataset)}                              {train_dataset.shape} {train_dataset.features}\n{train_dataset}")
print(f"train_dataset_tensor['input_ids']:             {type(train_dataset['input_ids'])}             len={len(train_dataset['input_ids'])}             shape={train_dataset['input_ids'].shape}            ") #\n{train_dataset['input_ids']}")
print(f"train_dataset_tensor['attention_mask']:        {type(train_dataset['attention_mask'])}        len={len(train_dataset['attention_mask'])}        shape={train_dataset['attention_mask'].shape}       ") #\n{train_dataset['attention_mask']}")
print(f"train_dataset_tensor['global_attention_mask']: {type(train_dataset['global_attention_mask'])} len={len(train_dataset['global_attention_mask'])} shape={train_dataset['global_attention_mask'].shape}") #\n{train_dataset['global_attention_mask']}")
print(f"train_dataset_tensor['labels']:                {type(train_dataset['labels'])}                len={len(train_dataset['labels'])}                shape={train_dataset['labels'].shape}               ") #\n{train_dataset['labels']}")

train_dataset_tensor:                          <class 'datasets.arrow_dataset.Dataset'>                              (9600, 4) {'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None), 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None), 'global_attention_mask': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), 'labels': Sequence(feature=Value(dtype='float32', id=None), length=-1, id=None)}
Dataset({
    features: ['input_ids', 'attention_mask', 'global_attention_mask', 'labels'],
    num_rows: 9600
})
train_dataset_tensor['input_ids']:             <class 'torch.Tensor'>             len=9600             shape=torch.Size([9600, 1024])            
train_dataset_tensor['attention_mask']:        <class 'torch.Tensor'>        len=9600        shape=torch.Size([9600, 1024])       
train_dataset_tensor['global_attention_mask']: <class 'torch.Tensor'> len=9600 shape=torch.Size([9600, 1024])
train_dataset_tensor['labels

## Truncated part

In [41]:
def get_truncated_part(text):
  tokens = tokenizer(
      text,
      truncation                = True,
      padding                   = 'max_length',
      max_length                = max_length,
      return_overflowing_tokens = True,
      return_tensors            = None
  )
  print(f"tokens.keys(): {tokens.keys()}")

  # Get the truncated tokens
  truncated_ids = tokens["input_ids"][0]
  print(f"truncated_ids: {type(truncated_ids)} {truncated_ids}")
  #overflow_ids  = tokens["overflow_to_sample_mapping"][0]
  #print(f"overflow_ids: {type(overflow_ids)} {overflow_ids}")

  # Decode the tokens back to text
  truncated_text = tokenizer.decode(truncated_ids, skip_special_tokens=True)
  #overflow_text  = tokenizer.decode(overflow_ids, skip_special_tokens=True)

  print(f"original_text :\n{text}")
  print(f"truncated_text:\n{truncated_text}")
  #print(f"overflow_text:\n{overflow_text}")

  original_tokens  = tokenizer.tokenize(text)
  truncated_tokens = tokenizer.tokenize(truncated_text)
  #overflow_tokens  = tokenizer.tokenize(overflow_text)

  print(f"original_tokens count : {len(original_tokens)}")
  print(f"truncated_tokens count: {len(truncated_tokens)}")
  #print(f"overflow_tokens count: {len(overflow_tokens)}")

In [42]:
example_text = datasetDict['train'][0]['text']
#get_truncated_part(example_text)

In [43]:
inputs = tokenizer(
    example_text,
    truncation     = True,
    padding        = 'max_length',
    max_length     = max_length,
    return_tensors = 'pt'
)

## Forward pass for multi-label classification

In [44]:
outputs = model(
    input_ids      = inputs.input_ids,
    attention_mask = inputs.attention_mask
)

Initializing global attention on CLS token...


In [45]:
print(f"outputs: {type(outputs)} {outputs.keys()}\n{outputs}")

outputs: <class 'transformers.models.longformer.modeling_longformer.LongformerSequenceClassifierOutput'> odict_keys(['logits'])
LongformerSequenceClassifierOutput(loss=None, logits=tensor([[ 0.1566,  0.0795, -0.0896, -0.0404, -0.1394, -0.1068]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None, global_attentions=None)


In [46]:
# Logits (= raw model outputs)
logits = outputs.logits
print(f"logits: {type(logits)} {logits.shape}\n{logits}")

logits: <class 'torch.Tensor'> torch.Size([1, 6])
tensor([[ 0.1566,  0.0795, -0.0896, -0.0404, -0.1394, -0.1068]],
       grad_fn=<AddmmBackward0>)


In [47]:
# Convert logits to probabilities
sigmoid = torch.nn.Sigmoid()
probs   = sigmoid(logits)
print(f"probs: {type(probs)} {probs.shape}\n{probs}")

probs: <class 'torch.Tensor'> torch.Size([1, 6])
tensor([[0.5391, 0.5199, 0.4776, 0.4899, 0.4652, 0.4733]],
       grad_fn=<SigmoidBackward0>)


In [48]:
example = encoded_dataset['train'][0]

In [49]:
print(f"example: {type(example)} {example.keys()}\n{example}")
print()
#print(f"example['input_ids']: {type(example['input_ids'])} {len(example['input_ids'])}\n{example['input_ids']}")
#print(f"example['attention_mask']: {type(example['attention_mask'])} {len(example['attention_mask'])}\n{example['attention_mask']}")
#print(f"example['labels']:  {type(example['labels'])} {len(example['labels'])}\n{example['labels']}")

example: <class 'dict'> dict_keys(['input_ids', 'attention_mask', 'global_attention_mask', 'labels'])
{'input_ids': tensor([    0, 10653,  1178,  ...,     1,     1,     1]), 'attention_mask': tensor([1, 1, 1,  ..., 0, 0, 0]), 'global_attention_mask': tensor([1, 0, 0,  ..., 0, 0, 0]), 'labels': tensor([0., 0., 1., 1., 0., 0.])}



In [50]:
tokenizer.decode(example['input_ids'])

"<s>Proximus SpearIT - Senior Web Security Engineer   F5, Security, Firewall, LTM, ASM   Proximus SpearIT   Job description Install, deploy, manage and operate security solutions based on Web Application Firewall systems Mandatory knowledge of F5 Technologies (LTM and ASM) Good Knowledge of Firewall, Proxy, Networking (Routing & Switching) systems and technologies Testing of new configuration, equipment and releases Reporting on network/security usage and performance Participate to project and deployment of new security architecture Deliver technical solutions and improve the level of protection of the system Develop the technical documentation regarding the operational procedures Be part in the security incident response process and participate to On-Duty staffing outside business hours. Job requirements Advanced skills in F5 (LTM and ASM) architectures, solutions and products Proven professional experience of several years (>4 years) in a similar operational environment Master's degr

In [51]:
[id2label[idx] for idx, label in enumerate(example['labels']) if label == 1.0]

['137', '138']

## Set PyTorch format to ensures correctness and compatibility with PyTorch pipelines
The 3 Hugging Face Dataset are formatted as PyTorch Dataset.

In [52]:
encoded_dataset.set_format('torch')

## Workflow

- 3 steps: training, evaluation, prediction
- 3 datasets: train, validation, test
- 3 Trainer functions: train, evaluate, predict
---
* training uses train_dataset
* evaluation uses validation_dataset
* test uses test_dataset

## Training step


In [53]:
batch_size  = batch_size
metric_name = "f1"

In [54]:
print(f"input_ids:              {type(encoded_dataset['train']['input_ids'][0])}\t{encoded_dataset['train']['input_ids'][0].shape}")
print(f"attention_mask:         {type(encoded_dataset['train']['attention_mask'][0])}\t{encoded_dataset['train']['attention_mask'][0].shape}")
print(f"global_attention_mask:  {type(encoded_dataset['train']['global_attention_mask'][0])}\t{encoded_dataset['train']['global_attention_mask'][0].shape}")
print(f"labels:                 {type(encoded_dataset['train'][0]['labels'])}\t{encoded_dataset['train'][0]['labels'].shape}")

input_ids:              <class 'torch.Tensor'>	torch.Size([1024])
attention_mask:         <class 'torch.Tensor'>	torch.Size([1024])
global_attention_mask:  <class 'torch.Tensor'>	torch.Size([1024])
labels:                 <class 'torch.Tensor'>	torch.Size([6])


### Execute a forward pass for debugging or verification purposes (cf. BERT_3_1 in Notion BERT database)

In [55]:
outputs = model(
    input_ids      = encoded_dataset['train']['input_ids'][0].unsqueeze(0),
    attention_mask = encoded_dataset['train']['attention_mask'][0].unsqueeze(0),
    labels         = encoded_dataset['train'][0]['labels'].unsqueeze(0)
)

In [56]:
print(f"outputs: {type(outputs)} {outputs.keys()}\n{outputs}")

outputs: <class 'transformers.models.longformer.modeling_longformer.LongformerSequenceClassifierOutput'> odict_keys(['loss', 'logits'])
LongformerSequenceClassifierOutput(loss=tensor(0.7046, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), logits=tensor([[ 0.1566,  0.0795, -0.0896, -0.0404, -0.1394, -0.1068]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None, global_attentions=None)


### Weighted loss function

In [57]:
%%capture

# Define the weighted loss function

class_weights = torch.tensor([7.68, 2.15, 0.61, 0.47, 0.68, 6.26], dtype=torch.float32).to(device)
loss_fn       = BCEWithLogitsLoss(pos_weight=class_weights)  # For multi-label classification (binary classification per label)

## Class supports, class weigths, weighted loss function

#Reminder:
#*   df_jobs      : <class 'pandas.core.frame.DataFrame'>
#*   df_jobs['id']: <class 'pandas.core.series.Series'>

#dataset = Dataset.from_pandas(df_jobs)
#*   dataset      : <class 'datasets.arrow_dataset.Dataset'>
#*   dataset['id']: <class 'list'>

#*   dataset_dict_jobs : <class 'datasets.dataset_dict.DatasetDict'>
#*   train_dataset     : <class 'datasets.arrow_dataset.Dataset'>
#*   validation_dataset: <class 'datasets.arrow_dataset.Dataset'>
#*   test_dataset      : <class 'datasets.arrow_dataset.Dataset'>


#We calculate the class supports for the train, validation and test datasets; the class weights and the weighted loss function are used for training only; the class supports of validation_dataset and test_dataset are calculated for information only.

def get_train_class_weights(datasetDict, labels):
  print(f"datasetDict: {type(datasetDict)} shape={datasetDict.shape}\n{datasetDict}")
  print(f"labels: {type(labels)} len={len(labels)}\n{labels}")

  dataset_train      = datasetDict['train']
  dataset_validation = datasetDict['validation']
  dataset_test       = datasetDict['test']

  def calculate_class_supports(dataset, labels):
    class_supports = dataset.map(
        lambda example: {col: example[col] for col in labels},
        batched=True
    ).to_pandas()[labels].sum(axis=0)
    return class_supports

  class_supports = {}

  for split_name, split_dataset in datasetDict.items():
    class_supports[split_name] = calculate_class_supports(split_dataset, labels)

  for split_name, split_class_supports in class_supports.items():
    print(f"{split_name}: {type(split_class_supports)} len={len(split_class_supports)}\n{split_class_supports}")

  train_class_supports_list = class_supports['train'].tolist()
  print(f"train_class_supports_list: {type(train_class_supports_list)} len={len(train_class_supports_list)} {train_class_supports_list}")

  train_class_supports_tensor = torch.tensor(train_class_supports_list, dtype=torch.float32)
  print(f"train_class_supports_tensor: {type(train_class_supports_tensor)} len={len(train_class_supports_tensor)} {train_class_supports_tensor}")

  train_total_samples = dataset_train.num_rows
  print(f"train_total_samples: {train_total_samples}")

  number_of_classes = len(labels)
  print(f"number_of_classes: {number_of_classes}")

  train_class_weights = train_total_samples / (number_of_classes * train_class_supports_tensor)
  print(f"train_class_weights: {type(train_class_weights)} len={len(train_class_weights)} {train_class_weights}")

  train_class_weights_sum = train_class_weights.sum()
  print(f"train_class_weights_sum: {train_class_weights_sum}")

  normalized_train_class_weights = (train_class_weights / train_class_weights_sum) * number_of_classes
  print(f"normalized_train_class_weights: {type(normalized_train_class_weights)} len={len(normalized_train_class_weights)} {normalized_train_class_weights}")

  # Positives samples per label
  supports = train_class_supports_tensor
  print(f"supports: {type(supports)} {len(supports)} {supports}")

  # Negatives samples per label
  negatives = train_total_samples - supports
  print(f"negatives: {type(negatives)} {len(negatives)} {negatives}")

  # pos_weights = negative to positive ratios
  pos_weights = negatives/supports
  print(f"pos_weights: {type(pos_weights)} {len(pos_weights)} {pos_weights}")

  # Normalize using min-max scaling
  normalized_pos_weights_minmax = (pos_weights - pos_weights.min()) / (pos_weights.max() - pos_weights.min())
  print(f"normalized_pos_weights_minmax: {type(normalized_pos_weights_minmax)} {len(normalized_pos_weights_minmax)} {normalized_pos_weights_minmax}")

  # Normalize using z-score standardization
  normalized_pos_weights_zscore = (pos_weights - pos_weights.mean()) / pos_weights.std()
  print(f"normalized_pos_weights_zscore: {type(normalized_pos_weights_zscore)} {len(normalized_pos_weights_zscore)} {normalized_pos_weights_zscore}")

  # Normalize using min-max scaling
  normalized_pos_weights_minmax = (pos_weights - pos_weights.min()) / (pos_weights.max() - pos_weights.min())
  print(f"normalized_pos_weights_minmax: {type(normalized_pos_weights_minmax)} {len(normalized_pos_weights_minmax)} {normalized_pos_weights_minmax}")

  # Normalize using z-score standardization
  normalized_pos_weights_zscore = (pos_weights - pos_weights.mean()) / pos_weights.std()
  print(f"normalized_pos_weights_zscore: {type(normalized_pos_weights_zscore)} {len(normalized_pos_weights_zscore)} {normalized_pos_weights_zscore}")

  # Normalize using sum-to-one
  normalized_pos_weights_sum1 = pos_weights / pos_weights.sum()
  print(f"normalized_pos_weights_sum1: {type(normalized_pos_weights_sum1)} {len(normalized_pos_weights_sum1)} {normalized_pos_weights_sum1}")

  return normalized_pos_weights_minmax
  #return normalized_pos_weights_zscore
  #return normalized_pos_weights_sum1

pos_weights = get_train_class_weights(datasetDict, labels)

loss_fn = BCEWithLogitsLoss(pos_weight=pos_weights.to(device))  # For multi-label classification (binary classification per label)
print(f"loss_fn: {type(loss_fn)} {loss_fn}")


In [58]:
def get_class_weights(labels=encoded_dataset['train']['labels']):
  print(f"labels: {type(labels)} len={len(labels)} shape={labels.shape}\n{labels}")

  num_samples, num_labels = labels.shape
  print(f"num_samples: {type(num_samples)} {num_samples}")
  print(f"num_labels:  {type(num_labels)}  {num_labels}")

  class_counts = labels.sum(dim=0)
  print(f"class_counts: {type(class_counts)} len={len(class_counts)}\n{class_counts}")

  pos_weights = (num_samples-class_counts) / class_counts
  print(f"pos_weights: {type(pos_weights)} len={len(pos_weights)}\n{pos_weights}")

  normalized_pos_weights_minmax = (pos_weights - pos_weights.min()) / (pos_weights.max() - pos_weights.min())
  print(f"normalized_pos_weights_minmax: {type(normalized_pos_weights_minmax)} {len(normalized_pos_weights_minmax)} {normalized_pos_weights_minmax}")

  normalized_pos_weights_zscore = (pos_weights - pos_weights.mean()) / pos_weights.std()
  print(f"normalized_pos_weights_zscore: {type(normalized_pos_weights_zscore)} {len(normalized_pos_weights_zscore)} {normalized_pos_weights_zscore}")

  normalized_pos_weights_sum1 = pos_weights / pos_weights.sum()
  print(f"normalized_pos_weights_sum1: {type(normalized_pos_weights_sum1)} {len(normalized_pos_weights_sum1)} {normalized_pos_weights_sum1}")

  #return pos_weights
  #return normalized_pos_weights_minmax
  #return normalized_pos_weights_zscore
  return normalized_pos_weights_sum1

In [59]:
pos_weights = get_class_weights()

labels: <class 'torch.Tensor'> len=9600 shape=torch.Size([9600, 6])
tensor([[0., 0., 1., 1., 0., 0.],
        [1., 1., 0., 0., 0., 1.],
        [0., 1., 1., 1., 0., 0.],
        ...,
        [0., 0., 1., 1., 1., 0.],
        [0., 0., 1., 1., 1., 0.],
        [0., 0., 0., 1., 1., 0.]])
num_samples: <class 'int'> 9600
num_labels:  <class 'int'>  6
class_counts: <class 'torch.Tensor'> len=6
tensor([ 554., 1789., 6639., 8754., 6055.,  715.])
pos_weights: <class 'torch.Tensor'> len=6
tensor([16.3285,  4.3661,  0.4460,  0.0966,  0.5855, 12.4266])
normalized_pos_weights_minmax: <class 'torch.Tensor'> 6 tensor([1.0000, 0.2630, 0.0215, 0.0000, 0.0301, 0.7596])
normalized_pos_weights_zscore: <class 'torch.Tensor'> 6 tensor([ 1.5167, -0.1917, -0.7515, -0.8014, -0.7316,  0.9595])
normalized_pos_weights_sum1: <class 'torch.Tensor'> 6 tensor([0.4768, 0.1275, 0.0130, 0.0028, 0.0171, 0.3628])


In [60]:
loss_fn = BCEWithLogitsLoss(pos_weight=pos_weights.to(device))

### FocalLoss

In [61]:
class FocalLoss(Module):
  """
  Focal Loss implementation
  """
  def __init__(self, alpha=1.0, gamma=2.0, logits=False, reduce=True):
    super(FocalLoss, self).__init__()
    self.alpha   = alpha
    self.gamma   = gamma
    self.logits  = logits  # This flag is to indicate whether input is logits or probability
    self.reduce  = reduce

  # inputs  = model's predictions: PyTorch tensor, shape=(batch_size, num_classes)
  # targets = ground truth labels: PyTorch tensor, shape=same as inputs shape
  def forward(self, inputs, targets):
    # Here, we check if input is probability or logits
    if self.logits:
      # Input is logits
      BCE_loss = F.binary_cross_entropy_with_logits(inputs, targets, reduction='none')
    else:
      # Input is probability
      BCE_loss = F.binary_cross_entropy(inputs, targets, reduction='none')
    pt = torch.exp(-BCE_loss)
    F_loss = self.alpha * (1-pt)**self.gamma * BCE_loss

    if self.reduce:
      return torch.mean(F_loss)
    else:
      return F_loss

  def __repr__(self):
    return f"FocalLoss(alpha={self.alpha}, gamma={self.gamma}, logits={self.logits}, reduce={self.reduce})"

  def __str__(self):
    return f"FocalLoss(alpha={self.alpha}, gamma={self.gamma}, logits={self.logits}, reduce={self.reduce})"

  def __call__(self, inputs, targets):
    return self.forward(inputs, targets)

In [62]:
focal_loss_fn = FocalLoss(alpha=0.5, gamma=4.0, logits=True, reduce=True)
print(f"focal_loss_fn: {type(focal_loss_fn)} {focal_loss_fn}")

focal_loss_fn: <class '__main__.FocalLoss'> FocalLoss(alpha=0.5, gamma=4.0, logits=True, reduce=True)


### Training Metrics
  source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/

In [63]:
def multi_label_metrics_OLD(logits, true_labels):
  """
  logits => sigmoid => probabilities => predictions

  Parameters:
  - logits     : raw, unnormalized scores outputted by the model  (numpy array of shape (batch_size, num_labels))
  - true_labels: actual labels for the data                       (numpy array of shape (batch_size, num_labels))

  Returns:
  - metrics: dictionary of scores
  """
  average = 'macro'

  sigmoid = torch.nn.Sigmoid()
  probs   = sigmoid(torch.Tensor(logits))
  # next, use threshold to turn them into integer predictions
  preds = np.zeros(probs.shape)
  preds[np.where(probs > threshold)] = 1

  # compute metrics
  f1                   = f1_score               (y_true=true_labels, y_pred=preds,  average=average)    #, zero_division=1)
  precision            = precision_score        (y_true=true_labels, y_pred=preds,  average=average)    #, zero_division=1)
  recall               = recall_score           (y_true=true_labels, y_pred=preds,  average=average)    #, zero_division=1)
  roc_auc              = roc_auc_score          (y_true=true_labels, y_score=probs, average=average)
  precision_recall_auc = average_precision_score(y_true=true_labels, y_score=probs, average=average)
  accuracy             = accuracy_score         (y_true=true_labels, y_pred=preds)

  metrics = {
      'f1'                  : f1,
      'precision'           : precision,
      'recall'              : recall,
      'roc_auc'             : roc_auc,
      'precision_recall_auc': precision_recall_auc,
      'accuracy'            : accuracy
  }

  return metrics

In [64]:
def multi_label_metrics(logits, true_labels, threshold=0.5):
    """
    Compute multi-label classification metrics.

    Parameters:
    - logits      : raw, unnormalized scores from the model  (numpy array of shape (batch_size, num_labels))
    - true_labels : actual labels                            (numpy array of shape (batch_size, num_labels))
    - threshold   : decision threshold for converting probabilities to binary predictions

    Returns:
    - metrics: dictionary of scores

    average:
    - per sample: metrics are computed for each sample (for each instance, for each row of y_true and y_pred) and then averaged across all samples.
    - per label : metrics are computed for each label separately, and then averaged across all labels.
    - per bacht : metrics are computed for each batch, and then averaged across all batches.

    - 'micro'   : Calculate metrics globally by counting the total true positives, false negatives and false positives.
    - 'macro'   : Calculate metrics for each label separately, and find their unweighted mean.
                  This does not take label imbalance into account. This is fine for balanced datasets but not for imbalanced datasets
                  since rare labels are given equal weights.
                  Averages the metric for each label without considering their imbalance, without considering label frequency.
                  When to use: when wanting equal importance for all labels, including rare ones.
    - 'weighted': Calculate metrics for each label separately, and find their average weighted
                  by support (the number of true instances for each label). This alters ‘macro’ to account for label imbalance;
                  it can result in an F-score that is not between precision and recall.
                  Averages per label weighted by their support (the number of true instances for each label), without considering label frequency.
                  When to use: when wanting to reflect label imbalance (common labels contribute more).
    - 'samples' : Calculate metrics per sample instead of per label, and find their average
                  (only meaningful for multilabel-classification where this differs from accuracy_score).
                  Computes the metric per sample and then averages across all samples.
                  When to use: when each sample has multiple correct labels.
    """
    average = 'macro'  # 'micro' 'macro' 'weighted' 'samples'

    # Convert logits to probabilities
    probs = torch.sigmoid(torch.tensor(logits)).numpy()  # Keep everything in NumPy

    # Apply threshold to get predictions
    preds = (probs > threshold).astype(int)

    """
    zero_division=0:
    - only for f1, precision and recall because they involve division where the denominator can be zero: some labels might never
      be predicted (y_pred = 0 for all samples), or they might not appear in the true_labels (y_true = 0 for all samples)
    - ROC AUC             : works with probabilities and does not involve division by zero
    - Precision-Recall AUC: also based on ranking, so no zero division issue
    - Accuracy            : just compares exact matches, so no zero division issue
    """
    f1         = f1_score               (y_true=true_labels, y_pred=preds,  average=average, zero_division=0)
    precision  = precision_score        (y_true=true_labels, y_pred=preds,  average=average, zero_division=0)
    recall     = recall_score           (y_true=true_labels, y_pred=preds,  average=average, zero_division=0)
    roc_auc    = roc_auc_score          (y_true=true_labels, y_score=probs, average=average)
    pr_auc     = average_precision_score(y_true=true_labels, y_score=probs, average=average)
    subset_acc = accuracy_score         (true_labels, preds)  # Subset accuracy (requires exact match per sample)
    hamming    = hamming_loss           (true_labels, preds)  # Better for imbalanced multi-label tasks

    metrics = {
        'f1'          : f1,
        'precision'   : precision,
        'recall'      : recall,
        'roc_auc'     : roc_auc,
        'precision_recall_auc': pr_auc,
        'subset_accuracy': subset_acc,
        'hamming_loss': hamming
    }

    return metrics

In [65]:
# Evaluation batch per batch
def compute_metrics(p: EvalPrediction):
    #print(f"p.predictions: {type(p.predictions)} {p.predictions.shape}\n{p.predictions[:5]}")
    #print(f"p.label_ids: {type(p.label_ids)} {p.label_ids.shape}\n{p.label_ids[:5]}")

    preds  = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    result = multi_label_metrics(logits=preds,true_labels=p.label_ids)

    return result

### HF transformer Trainer and CustomTrainer
Abstracts the training loop.

In [66]:
training_args = TrainingArguments(
    output_dir                  = './training_results',  # where model predictions and checkpoints will be written during training
    overwrite_output_dir        = True,
    logging_dir                 = './logs',
    logging_steps               = 50,
    save_steps                  = 500,
    save_total_limit            = 2,
    eval_strategy               = 'epoch',
    save_strategy               = 'epoch',
    learning_rate               = learning_rate,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size  = batch_size,
    gradient_accumulation_steps = gradient_accumulation_steps,
    num_train_epochs            = epochs,
    weight_decay                = 0.01,
    load_best_model_at_end      = True,
    metric_for_best_model       = metric_name,
    run_name                    = run_name,
    fp16                        = fp,
    report_to                  = 'wandb'
)

In [67]:
class CustomTrainer(Trainer):

  def __init__(self, *args, loss_fn=None, **kwargs):
    super().__init__(*args, **kwargs)
    self.loss_fn = loss_fn

  """
  # No print in compute_loss because out of memory because prints are batch per batch
  def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):

    #print(f"inputs passed to compute_loss: {inputs.keys()}")
    #input_ids             = inputs['input_ids']                        # shape: batch_size, sequence_length
    #attention_mask        = inputs['attention_mask']                   # shape: batch_size, sequence_length
    #global_attention_mask = inputs.get('global_attention_mask', None)  # shape: batch_size, sequence_length; optional as LongFormer specific
    labels                = inputs.pop('labels', None)                 # shape: batch_size, num_labels; needed for loss computation, not required by the model

    #outputs = model(**inputs, global_attention_mask=global_attention_mask)  # Forward pass
    # Forward pass
    #outputs = model(
    #    input_ids             = input_ids,
    #    attention_mask        = attention_mask,
    #    global_attention_mask = global_attention_mask,
    #    labels                = labels
    #)
    outputs = model(**inputs, labels=labels)
    #print(f"outputs: {type(outputs)} {outputs.keys()}\n{outputs}")
    logits = outputs.logits  # shape: (batch_size, num_labels)

    # If labels are provided, compute loss
    if labels is not None:
      # Use the custom loss function if provided
      if self.loss_fn is not None:
        loss = self.loss_fn(logits, labels)  # Compute weighted loss
      else:
        # Default loss: BCEWithLogitsLoss
        loss_fn = BCEWithLogitsLoss()
        loss    = loss_fn(logits, labels)    # Compute loss
      return (loss, outputs) if return_outputs else loss

    # If no labels, return outputs only, for evaluation or prediction
    return outputs
    """
  def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
    labels  = inputs.pop('labels', None)
    outputs = model(**inputs, labels=labels)
    logits  = outputs.logits

    if labels is not None:
      if self.loss_fn is not None:
        loss = self.loss_fn(logits, labels)
      else:
        loss_fn = BCEWithLogitsLoss()
        loss    = loss_fn(logits, labels)
      return (loss, outputs) if return_outputs else loss

    return outputs

In [68]:
"""
trainer = CustomTrainer(
    model           = model,
    args            = training_args,
    train_dataset   = encoded_dataset["train"],
    eval_dataset    = encoded_dataset["validation"],
    compute_metrics = compute_metrics,
    loss_fn         = focal_loss_fn,
)
"""

trainer = Trainer(
    model           = model,
    args            = training_args,
    train_dataset   = encoded_dataset["train"],
    eval_dataset    = encoded_dataset["validation"],
    compute_metrics = compute_metrics,
)

### trainer.train

In [69]:
trainer_train_results = trainer.train()

print(f"trainer_train_results: {type(trainer_train_results)} len={len(trainer_train_results)}\n{trainer_train_results}")
print()
print(f"trainer_train_results.metrics: {type(trainer_train_results.metrics)} len={len(trainer_train_results.metrics)}\n{json.dumps(trainer_train_results.metrics, indent=4)}")

Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss,F1,Precision,Recall,Roc Auc,Precision Recall Auc,Subset Accuracy,Hamming Loss,Runtime,Samples Per Second,Steps Per Second
1,0.361100,0.365945,0.633061,0.761657,0.629506,0.765014,0.696603,0.331667,0.160833,18.047400,66.491000,8.311000
2,0.312500,0.339576,0.698218,0.706457,0.697626,0.812516,0.734258,0.389167,0.153889,17.997600,66.676000,8.334000
3,0.278800,0.326096,0.712794,0.764411,0.691806,0.820314,0.744163,0.435000,0.139028,18.012900,66.619000,8.327000
4,0.270100,0.311757,0.733706,0.762410,0.717188,0.835752,0.757555,0.462500,0.128194,17.992900,66.693000,8.337000
5,0.245400,0.311873,0.732027,0.764638,0.714829,0.835045,0.757770,0.468333,0.127083,18.077400,66.381000,8.298000


trainer_train_results: <class 'transformers.trainer_utils.TrainOutput'> len=3
TrainOutput(global_step=1500, training_loss=0.30800257809956866, metrics={'train_runtime': 2424.8824, 'train_samples_per_second': 19.795, 'train_steps_per_second': 0.619, 'total_flos': 3.1529784508416e+16, 'train_loss': 0.30800257809956866, 'epoch': 5.0})

trainer_train_results.metrics: <class 'dict'> len=6
{
    "train_runtime": 2424.8824,
    "train_samples_per_second": 19.795,
    "train_steps_per_second": 0.619,
    "total_flos": 3.1529784508416e+16,
    "train_loss": 0.30800257809956866,
    "epoch": 5.0
}


In [70]:
print("trainer.train successfully completed")

trainer.train successfully completed


### trainer.train results: save locally and upload to HF Hub

In [71]:
if upload_to_HF is True:

  name               = "trainer_train"
  trainer_train_path = f"{name}.json"

  with open(trainer_train_path, "w") as f:
    json.dump(trainer_train_results, f)

  print(f"{name} results successfully saved locally to {trainer_train_path}")

  upload_file(
      path_or_fileobj = trainer_train_path,
      path_in_repo    = trainer_train_path,
      repo_id         = repo_id,
      repo_type       = 'dataset',
      commit_message  = f"{name}_{timestamp}"
  )

  print(f"{name} results successfully uploaded to HF Hub as {trainer_train_path}")

### trainer.train results: check that the uploaded file can be downloaded
File locally downloaded to:
/root/.cache/huggingface/hub/datasets-claudelepere-skill_classification/snapshots/full_commit_hash/trainer_train_results.json

In [72]:
if upload_to_HF is True:

  file_path = hf_hub_download(repo_type="dataset", repo_id=repo_id, filename=trainer_train_path)

  print(f"file_path: {file_path}")

## Evaluation step

### Evaluation 1: trainer.evaluate
trainer.evaluate uses a fixed threshold of 0.5 to convert logits into binary labels, which is often suboptimal for imbalanced data.

In [73]:
evaluation_trainer_evaluate_results = trainer.evaluate(
    #eval_dataset=encoded_dataset["validation"],  # by default, trainer.evaluate() evaluates the dataset passed as eval_dataset during training
    metric_key_prefix="eval"                     # prefix for the evaluation metrics
)

print(f"evaluation_trainer_evaluate_results: {type(evaluation_trainer_evaluate_results)} len={len(evaluation_trainer_evaluate_results)}\n{json.dumps(evaluation_trainer_evaluate_results, indent=4)}")


evaluation_trainer_evaluate_results: <class 'dict'> len=12
{
    "eval_loss": 0.31175708770751953,
    "eval_f1": 0.733706198449156,
    "eval_precision": 0.7624103556083979,
    "eval_recall": 0.7171878946388442,
    "eval_roc_auc": 0.8357515899719302,
    "eval_precision_recall_auc": 0.7575550204299834,
    "eval_subset_accuracy": 0.4625,
    "eval_hamming_loss": 0.12819444444444444,
    "eval_runtime": 18.0531,
    "eval_samples_per_second": 66.471,
    "eval_steps_per_second": 8.309,
    "epoch": 5.0
}


In [ ]:
print("evaluation 1: trainer.evaluate: successfully completed")

### Evaluation 1: trainer.evaluate results: save locally and upload to HF Hub

In [ ]:
if upload_to_HF is True:

  name                             = "evaluation_trainer_evaluate"
  evaluation_trainer_evaluate_path = f"{name}.json"

  with open(evaluation_trainer_evaluate_path, "w") as f:
    json.dump(evaluation_trainer_evaluate_results, f)

  print(f"{name} successfully saved locally to {evaluation_trainer_evaluate_path}")

  upload_file(
      path_or_fileobj = evaluation_trainer_evaluate_path,
      path_in_repo    = evaluation_trainer_evaluate_path,
      repo_id         = repo_id,
      repo_type       = 'dataset',
      commit_message  = f"{name}_{timestamp}"
  )

  print(f"{name} successfully uploaded to HF Hub as {evaluation_trainer_evaluate_path}")

### Evaluation 2: trainer.predict

In [ ]:
def predict_with_optimized_thresholds(trainer, dataset, id2label, threshold_tuning=False, thresholds=None):
    """
    Predicts using trainer.predict(), with optional threshold tuning, using NumPy arrays, and not PyTorch tensors

    Parameters:
    - trainer                         : Hugging Face Trainer or CustomTrainer instance
    - dataset                         : Dataset to predict on
    - id2label                        : Dictionary mapping label indices (int) to label names (string)
    - threshold_tuning                : Boolean to enable threshold tuning per class
    - thresholds       (numpy.ndarray): Custom thresholds for classification

    Returns:
    if threshold_tuning is True:
        - best_thresholds      (numpy.ndarray): optimized threshold per class (= per label)
        - best_thresholds_dict (dict)         : optimized threshold per class (= per label)
        - best_metrics         (dict)         : best f1, precision, recall per class (= per label)
    else:
        - thresholds      (numpy.ndarray): fixed threshold per class (= per label)
        - thresholds_dict (dict)         : fixed threshold per class (= per label)
        - metrics         (dict)         : computed with provided thresholds or default to 0.5
    """
    # Predict
    predictions_output = trainer.predict(dataset)
    logits             = predictions_output.predictions
    true_labels        = predictions_output.label_ids

    # Convert logits to probabilities (with np, not with )
    probs = 1 / (1 + np.exp(-logits))  # Sigmoid function

    num_labels           = len(id2label)
    threshold_candidates = np.linspace(0.05, 0.95, 19)
    best_thresholds      = np.zeros(num_labels)

    best_metrics         = {label: {'f1': 0.0, 'precision': 0.0, 'recall': 0.0} for label in id2label.values()}     # if threshold_tuning is True
    metrics              = {label: {'f1': 0.0, 'precision': 0.0, 'recall': 0.0} for label in id2label.values()}     # else

    if threshold_tuning is True:
        # Vectorized computation for speed
        for label_idx, label in id2label.items():
            preds = probs[:, label_idx][:, None] > threshold_candidates  # Create a matrix of shape (num_samples num_thresholds)

            # Compute precision, recall, and F1 for all thresholds at once
            precision, recall, f1, _ = precision_recall_fscore_support(
                np.tile(true_labels[:, label_idx], (len(threshold_candidates), 1)).T, preds, average=None, zero_division=0
            )

            # Find the best threshold based on F1
            best_idx                   = np.argmax(f1)
            best_thresholds[label_idx] = threshold_candidates[best_idx]
            best_metrics[label]        = {'f1': f1[best_idx], 'precision': precision[best_idx], 'recall': recall[best_idx]}

        best_thresholds_dict = {id2label[i]: best_thresholds[i].item() for i in range(len(best_thresholds))}

        #print("==== best_thresholds, best_threshold_dict and best_metrics ====")
        #print(f"best_thresholds: {type(best_thresholds)} shape={best_thresholds.shape}\n{best_thresholds}")                     # <class 'numpy.ndarray'> shape=(6,)
        #print(f"best_thresholds_dict: {type(best_thresholds_dict)} len={len(best_thresholds_dict)}\n{best_thresholds_dict}")    # <class 'dict'> len=6
        #print(f"best_metrics:         {type(best_metrics)} len={len(best_metrics)}\n{json.dumps(best_metrics, indent=4)}")      # <class 'dict'> len=6
        #print("=========================")
        #print()

        return best_thresholds, best_thresholds_dict, best_metrics

    # Apply provided thresholds or default to 0.5
    elif thresholds is None:
        thresholds = np.full(num_labels, 0.5)

    for label_idx, label in id2label.items():
        preds = probs[:, label_idx][:, None] > thresholds  # Create a matrix of shape (num_samples num_thresholds)

        # Compute precision, recall, and F1 for all thresholds at once
        precision, recall, f1, _ = precision_recall_fscore_support(
            np.tile(true_labels[:, label_idx], (len(thresholds), 1)).T, preds, average=None, zero_division=0
        )

        metrics[label] = {'f1': f1[label_idx], 'precision': precision[label_idx], 'recall': recall[label_idx]}

    thresholds_dict = {label: threshold for label, threshold in zip(id2label.values(), thresholds.tolist())}

    #print("==== provided thresholds and metrics ====")
    #print(f"thresholds: {type(thresholds)} shape={thresholds.shape}\n{thresholds}")                     # <class 'numpy.ndarray'> shape=(6,)
    #print(f"thresholds_dict: {type(thresholds_dict)} len={len(thresholds_dict)}\n{thresholds_dict}")    # <class 'dict'> len=6
    #print(f"metrics   : {type(metrics)} len={len(metrics)}\n{json.dumps(metrics, indent=4)}")           # <class 'dict'> len=6
    #print("=========================")
    #print()

    return thresholds, thresholds_dict, metrics

In [86]:
thresholds, thresholds_dict, trainer_predict_metrics = predict_with_optimized_thresholds(trainer, validation_dataset, id2label, threshold_tuning=threshold_tuning, thresholds=None)

print(f"thresholds: {type(thresholds)} shape={thresholds.shape} {thresholds}")
print(f"thresholds_dict: {type(thresholds_dict)} len={len(thresholds_dict)}\n{thresholds_dict}")
print(f"trainer_predict_metrics: {type(trainer_predict_metrics)} len={len(trainer_predict_metrics)}\n{json.dumps(trainer_predict_metrics, indent=4)}")

#test_preds, test_true_labels = predict_with_optimized_thresholds(trainer, test_dataset, threshold_tuning=threshold_tuning, thresholds=best_thresholds, threshold=threshold)

thresholds: <class 'numpy.ndarray'> shape=(6,) [0.55 0.25 0.25 0.25 0.3  0.7 ]
thresholds_dict: <class 'dict'> len=6
{'135': 0.5499999999999999, '136': 0.25, '137': 0.25, '138': 0.25, '139': 0.3, '390': 0.7}
trainer_predict_metrics: <class 'dict'> len=6
{
    "135": {
        "f1": 0.6504065040650406,
        "precision": 0.7017543859649122,
        "recall": 0.6060606060606061
    },
    "136": {
        "f1": 0.6217821782178218,
        "precision": 0.5836431226765799,
        "recall": 0.6652542372881356
    },
    "137": {
        "f1": 0.8752136752136752,
        "precision": 0.8356909684439608,
        "recall": 0.9186602870813397
    },
    "138": {
        "f1": 0.9610738255033557,
        "precision": 0.9322916666666666,
        "recall": 0.9916897506925207
    },
    "139": {
        "f1": 0.80178671133445,
        "precision": 0.6838095238095238,
        "recall": 0.9689608636977058
    },
    "390": {
        "f1": 0.5899280575539568,
        "precision": 0.7068965517241379

In [ ]:
print("evaluation 2: trainer.predict: successfully completed")

### Evaluation 1: trainer.predict results: save locally and upload to HF Hub


In [ ]:
if upload_to_HF is True:

  name                             = "evaluation_trainer_predict"
  evaluation_trainer_predict_path = f"{name}.json"

  with open(evaluation_trainer_predict_path, "w") as f:
    json.dump(trainer_predict_metrics, f)

  print(f"{name} successfully saved locally to {evaluation_trainer_predict_path}")

  upload_file(
      path_or_fileobj = evaluation_trainer_predict_path,
      path_in_repo    = evaluation_trainer_predict_path,
      repo_id         = repo_id,
      repo_type       = 'dataset',
      commit_message  = f"{name}_{timestamp}"
  )

  print(f"{name} successfully uploaded to HF Hub as {evaluation_trainer_predict_path}")

### Evaluation 2: best thresholds: save locally (as a dict) and upload to HF Hub (as a JSON file in repo 'model')
best_thresholds: <class 'numpy.ndarray'> shape=(6,) but JSON doesn't support Numpy types

In [ ]:
if threshold_tuning is True and upload_to_HF is True:
    best_thresholds_dict = thresholds_dict

    name                 = "best_thresholds"
    best_thresholds_path = f"{name}.json"

    with open(best_thresholds_path, "w") as f:
        json.dump(best_thresholds_dict, f, indent=4)

    print(f"{best_thresholds_dict} successfully saved locally to {best_thresholds_path}")

    path_in_repo = "tuned_thresholds.json"

    upload_file(
        path_or_fileobj = best_thresholds_path,
        path_in_repo    = path_in_repo,
        repo_id         = repo_id,
        repo_type       = 'model',
        commit_message  = f"tuned_thresholds_{timestamp}"
        )

    print(f"{name} successfully uploaded to HF Hub as {path_in_repo}")

### Evaluation 3: model.eval

In [ ]:
def compute_metrics_with_threshold(model, dataset, tuned_thresholds, id2label, batch_size=8):
    """
    Compute metrics during evaluation or test, by applying tuned thresholds

    Parameters:
    - model                                           : Hugging Face model
    - dataset                                         : Dataset to predict on
    - tuned_thresholds (list or NumPy array of floats): Tuned thresholds for each label
    - id2label                                        : Dictionary mapping label indices (int) to label names (string)
    - batch_size                                      : Batch size for prediction. Defaults to 8
    Returns:
    - metrics (dict)

    Compute metrics during evaluation or test, by applying tuned thresholds

    average:
    - 'micro'   : gives more weight to frequent labels     → best for imbalanced datasets where frequent labels are more important
    - 'macro'   : treats all labels equally                → best when you care about rare labels as much as frequent ones
    - 'weighted': like macro but considers label frequency → best if you want a compromise between macro and micro

    - 'macro' or 'weighted' AUC is often best because AUC isn't as affected by class imbalance as F1/Precision/Recall
    - 'macro'      AUC: usually the best because it treats all labels equally, avoiding the dominance of frequent labels
    - 'weighted'   AUC: similar to macro but considers label frequency
    - 'macro'   PR AUC: best for imbalanced datasets because it treats rare labels fairly
    - 'weighted PR AUC: also good, but slightly biased toward frequent labels

    PR AUC is better than ROC AUC when you care about positive examples in imbalanced data
    """
    dataloader = DataLoader(dataset, batch_size=batch_size)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)    # Move model to GPU/CPU
    model.eval()        # Set model to evaluation mode

    all_logits, all_labels = [], []

    with torch.no_grad():
        for batch in dataloader:
            labels  = batch.pop('labels')                           # Keep labels on CPU
            inputs  = {k: v.to(device) for k, v in batch.items()}   # Move inputs to device
            outputs = model(**inputs)

            all_logits.append(outputs.logits.cpu())     # Keep logits as tensors, move to CPU
            all_labels.append(labels)                   # Labels remain on CPU

    # Stack tensors
    logits = torch.cat(all_logits, dim=0).to(device)   # shape = (num_samples, num_labels), move to device
    labels = torch.cat(all_labels, dim=0).to(device)   # shape = (num_samples, num_labels), move to device

    # Convert logits to probabilities
    probs = torch.sigmoid(logits)           # shape = (num_samples, num_labels)

    # Apply per-class tuned thresholds (element-wise comparison)
    thresholds = torch.tensor(tuned_thresholds, dtype=torch.float32, device=device)     # Convert tuned_thresholds to tensor
    preds      = (probs > thresholds).int()                                             # Convert to binary predictions (1 or 0)

    # Compute TP, FP, FN, TN
    TP = ((preds == 1) & (labels == 1)).sum(dim=0).float()
    TN = ((preds == 0) & (labels == 0)).sum(dim=0).float()
    FP = ((preds == 1) & (labels == 0)).sum(dim=0).float()
    FN = ((preds == 0) & (labels == 1)).sum(dim=0).float()

    # Compute per-class metrics
    precision_per_class = TP / (TP + FP + 1e-8)
    recall_per_class    = TP / (TP + FN + 1e-8)
    f1_per_class        = 2 * (precision_per_class * recall_per_class) / (precision_per_class + recall_per_class + 1e-8)

    # Compute macro-averaged metrics
    macro_precision = precision_per_class.mean()
    macro_recall    = recall_per_class.mean()
    macro_f1        = f1_per_class.mean()

    # Compute accuracy
    accuracy = (preds == labels).float().mean()

    # Convert to NumPy for ROC-AUC and PR-AUC
    labels_np = labels.cpu().numpy()  # Move to CPU before converting
    probs_np  = probs.cpu().numpy()   # Move to CPU before converting

    # Compute ROC-AUC and PR-AUC
    roc_auc_macro = torch.tensor(roc_auc_score(labels_np, probs_np, average='macro', multi_class='ovr'))
    pr_auc_macro  = torch.tensor(average_precision_score(labels_np, probs_np, average='macro'))

    # Convert predictions to Numpy for classification_report
    preds_np = preds.cpu().numpy()  # Move to CPU before converting

    # Generate classification report
    class_names  = [id2label[i] for i in range(len(id2label))]
    class_report = classification_report(labels_np, preds_np, target_names=class_names, zero_division=0)

    print(f"\nClassification Report:\n{class_report}")

    # Store metrics
    metrics = {
        'accurary'             : accuracy.item(),
        'macro_precision'      : macro_precision.item(),
        'macro_recall'         : macro_recall.item(),
        'macro_f1'             : macro_f1.item(),
        'roc_auc_macro'        : roc_auc_macro.item(),
        'pr_auc_macro'         : pr_auc_macro.item(),
        'per_class_precision'  : {id2label[i]: precision_per_class[i].item() for i in range(len(id2label))},
        'per_class_recall'     : {id2label[i]: recall_per_class[i].item() for i in range(len(id2label))},
        'per_class_f1'         : {id2label[i]: f1_per_class[i].item() for i in range(len(id2label))},
        'classification_report': class_report,
    }

    return metrics

In [87]:
model_eval_metrics = compute_metrics_with_threshold(model, validation_dataset, thresholds, id2label, batch_size=16)

except_report = {k: v for k, v in model_eval_metrics.items() if k!='classification_report'}
report        = model_eval_metrics['classification_report']
print(f"model_eval_metrics: {type(except_report)} len={len(except_report)}\n{json.dumps(except_report, indent=4)}")
print(f"model_eval_metrics['classification_report']: {type(report)} len={len(report)}\n{report}")


Classification Report:
              precision    recall  f1-score   support

         135       0.70      0.61      0.65        66
         136       0.58      0.67      0.62       236
         137       0.84      0.92      0.88       836
         138       0.93      0.99      0.96      1083
         139       0.68      0.97      0.80       741
         390       0.71      0.51      0.59        81

   micro avg       0.80      0.92      0.85      3043
   macro avg       0.74      0.78      0.75      3043
weighted avg       0.81      0.92      0.86      3043
 samples avg       0.81      0.94      0.85      3043

model_eval_metrics: <class 'dict'> len=9
{
    "accurary": 0.8677778244018555,
    "macro_precision": 0.7406810522079468,
    "macro_recall": 0.7761330604553223,
    "macro_f1": 0.7500318288803101,
    "roc_auc_macro": 0.8357634090567586,
    "pr_auc_macro": 0.7579455976563922,
    "per_class_precision": {
        "135": 0.7017543911933899,
        "136": 0.5836431384086609,
 

In [ ]:
print("evaluation 3: model.eval: successfully completed")

### Evaluation 1: model.eval results: save locally and upload to HF Hub

In [ ]:
if upload_to_HF is True:

  name                       = "evaluation_model_eval"
  evaluation_model_eval_path = f"{name}.json"

  with open(evaluation_model_eval_path, "w") as f:
    json.dump(model_eval_metrics, f)

  print(f"{name} successfully saved locally to {evaluation_model_eval_path}")

  upload_file(
      path_or_fileobj = evaluation_model_eval_path,
      path_in_repo    = evaluation_model_eval_path,
      repo_id         = repo_id,
      repo_type       = 'dataset',
      commit_message  = f"{name}_{timestamp}"
  )

  print(f"{name} successfully uploaded to HF Hub as {evaluation_model_eval_path}")

In [88]:
raise Exception("I stop here")

Exception: I stop here

## Prediction step

### Prediction: trainer.evaluate

In [ ]:
prediction_trainer_evaluate_results = trainer.evaluate(
    eval_dataset=encoded_dataset['test'],  # by default, trainer.evaluate() evaluates the dataset passed as eval_dataset during training
    metric_key_prefix='test'
)

print(f"prediction_trainer_evaluate_results: {type(prediction_trainer_evaluate_results)} len={len(prediction_trainer_evaluate_results)}\n{json.dumps(prediction_trainer_evaluate_results, indent=4)}")


In [ ]:
print("prediction_trainer.evaluate successfully completed")

### Prediction: trainer.evaluate results: save locally and upload to HF Hub

In [ ]:
if upload_to_HF is True:

  name                             = "prediction_trainer_evaluate"
  prediction_trainer_evaluate_path = f"{name}.json"

  with open(prediction_trainer_evaluate_path, "w") as f:
    json.dump(prediction_trainer_evaluate_results, f)

  print(f"{name} results successfully saved locally to {prediction_trainer_evaluate_path}")

  upload_file(
      path_or_fileobj = prediction_trainer_evaluate_path,
      path_in_repo    = prediction_trainer_evaluate_path,
      repo_id         = repo_id,
      repo_type       = 'dataset',
      commit_message  = f"{name}_{timestamp}"
  )

  print(f"{name} results successfully uploaded to HF Hub as {prediction_trainer_evaluate_path}")

### Prediction: trainer.predict

In [ ]:
prediction_trainer_predict_results = predict_with_thresholds(trainer, test_dataset, id2label, threshold_tuning=False, thresholds=best_thresholds)

except_report = {k: v for k, v in prediction_trainer_predict_results.items() if k!='classification_report'}
report        = prediction_trainer_predict_results['classification_report']
print(f"prediction_trainer_predict_results: {type(except_report)} len={len(except_report)}\n{json.dumps(except_report, indent=4)}")
print(f"prediction_trainer_predict_results['classification_report']: {type(report)} len={len(report)}\n{report}")


In [ ]:
print("prediction_trainer.predict successfully completed")

### Prediction: trainer.predict results: save locally and upload to HF Hub

In [ ]:
if upload_to_HF is True:

  name                            = "prediction_trainer_predict"
  prediction_trainer_predict_path = f"{name}.json"

  with open(prediction_trainer_predict_path, "w") as f:
    json.dump(prediction_trainer_predict_results, f)

  print(f"{name} results successfully saved locally to {prediction_trainer_predict_path}")

  upload_file(
      path_or_fileobj = prediction_trainer_predict_path,
      path_in_repo    = prediction_trainer_predict_path,
      repo_id         = repo_id,
      repo_type       = 'dataset',
      commit_message  = f"{name}_{timestamp}"
  )

  print(f"{name} results successfully uploaded to HF Hub as {prediction_trainer_evaluate_path}")

## Upload tokenizer and model to HF Hub and check

In [ ]:
if upload_to_HF is True:

  # Upload
  commit_message = f"'model_tokenizer'_{timestamp}"

  tokenizer.push_to_hub(repo_id, commit_message=commit_message)  # commit_message as named parameter
  model.push_to_hub(    repo_id, commit_message=commit_message)  # commit_message as named parameter

  print(f"tokenizer and model successfully uploaded to HF Hub at {repo_id}")

  # Check
  def check_upload(repo_id):
    print()
    print("Tokenizer")
    tokenizer = LongformerTokenizerFast.from_pretrained(repo_id)
    print()
    print("Model")
    model = LongformerForSequenceClassification.from_pretrained(repo_id)
    print()

    inputs = tokenizer("Hello, my dog is cute", return_tensors="pt", truncation=True, padding=True)
    outputs = model(**inputs)

    print(f"outputs: {type(outputs)} {outputs.keys()}\n{outputs}")

  # To check if the upload was successful, download the tokenizer and the model
  check_upload(repo_id)

In [ ]:
print("It's the end")

In [ ]:
raise Exception("I stop here")

===========================================================================================================

In [ ]:
example = datasetDict['test'][0]
print(f"datasetDict['test'][0]: {type(example)} {example.keys()}\n{example}")

In [ ]:
inputs = tokenizer(
    example['text'],
    truncation     = True,
    padding        = 'max_length',
    max_length     = max_length,
    return_tensors = 'pt'
)

In [ ]:
with torch.no_grad():  # Disable gradient calculation during prediction
    outputs = model(
        input_ids=inputs.input_ids.to(device),
        attention_mask=inputs.attention_mask.to(device)
    )


In [ ]:
probs = torch.sigmoid(outputs.logits)


In [ ]:
best_thresholds = [0.4, 0.3, 0.3, 0.3, 0.4, 0.5]
preds = np.zeros_like(probs)  # Initialize predictions array
for label_idx in range(num_labels):
  preds[:, label_idx] = (probs[:, label_idx] > best_thresholds[label_idx])  #.astype(int)

In [ ]:
print(f"probs: {type(probs)} shape={probs.shape}\n{probs}")
print(f"preds: {type(preds)} shape={preds.shape}\n{preds}")

In [ ]:
print(f"labels: {type(labels)} {len(labels)}\n{labels}")
# '390': False, '135': False, '136': True, '137': True, '138': True, '139': False}

In [ ]:
%%capture

def tune_thresholds(true_labels, probs, id2label):
  """
  Tune thresholds for each label to maximize F1 alone, as F1 balances precision and recall into a single metric.

  Args:
    true_labels: actual labels for the data                                      (numpy array of shape (num_samples, num_labels))
    probs      : predicted probabilities                                         (numpy array of shape (num_samples, num_labels))
    id2label   : dictionary mapping label indices (int) to label names (string)

  Returns:
    best_thresholds: best threshold for each label                                                      (numpy array of shape (num_labels,))
    best_metrics   : dictionary of best F1, precision_for_best_f1 and recall_for_best_f1 for each label (dictionary of numpy arrays)
  """
  thresholds      = np.linspace(0.1, 0.9, 9)
  best_thresholds = np.zeros(len(id2label))
  best_metrics    = {label: {'f1': 0.0, 'precision': 0.0, 'recall': 0.0} for label in id2label.values()}

  for label_idx, label in id2label.items():
    for threshold in thresholds:
      pred                     = (probs[:, label_idx] > threshold).astype(int)
      precision, recall, f1, _ = precision_recall_fscore_support(true_labels[:, label_idx], pred, average='binary', zero_division=0)
      if f1 > best_metrics[label]['f1']:
        best_thresholds[label_idx]       = threshold
        best_metrics[label]['f1']        = f1
        best_metrics[label]['precision'] = precision
        best_metrics[label]['recall']    = recall

  print("==== tune_thresholds ====")
  print(f"best_thresholds: {type(best_thresholds)} shape={best_thresholds.shape}\n{best_thresholds}")
  print(f"best_metrics   : {type(best_metrics)}    len={len(best_metrics)}      \n{json.dumps(best_metrics, indent=4)}")
  print("=========================")
  print()

  return best_thresholds, best_metrics

In [ ]:
%%capture

def compute_metrics_with_threshold_SAV(probs, label_ids, thresholds, id2label):

  #Compute metrics during evaluation or test, by applying tuned thresholds

  #logits  = eval_preds.predictions
  #labels  = eval_preds.label_ids
  #sigmoid = torch.nn.Sigmoid  # Sigmoid or numpy?
  #probs   = sigmoid(logits).cpu().numpy()
  preds   = np.zeros_like(probs)

  if threshold_tuning:
    # Apply per-label tuned threshold
    for label_idx in id2label.keys():
        preds[:, label_idx] = (probs[:, label_idx] > thresholds[label_idx]).astype(int)
  else:
    # threhsolds = None, apply a fixed threshold to all labels
    for label_idx in id2label.keys():
        preds[:, label_idx] = (probs[:, label_idx] > threshold).astype(int)

  # Compute metrics
  f1                    = f1_score               (label_ids, preds, average='micro')
  precision             = precision_score        (label_ids, preds, average='micro')
  recall                = recall_score           (label_ids, preds, average='micro')
  accuracy              = accuracy_score         (label_ids, preds)
  roc_auc               = roc_auc_score          (label_ids, probs, average='micro')  # multi_class="ovr" <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
  precision_recall_auc  = average_precision_score(label_ids, probs, average='micro')

  # Use id2label for target_names
  report = classification_report(label_ids, preds, target_names=id2label.values(), zero_division=0)

  if threshold_tuning:
    _thresholds = thresholds.tolist()
  else:
    _thresholds = threshold

  metrics = {
      'f1'                   : f1,
      'precision'            : precision,
      'recall'               : recall,
      'accuracy'             : accuracy,
      'roc_auc'              : roc_auc,
      'precision_recall_auc' : precision_recall_auc,
      'thresholds'           : _thresholds,
      'classification_report': report
  }

  return metrics

In [ ]:
%%capture

def compute_metrics_with_threshold_OLD(probs, label_ids, thresholds, id2label):
    """
    Compute metrics during evaluation or test, by applying tuned thresholds

    average:
    - 'micro'   : gives more weight to frequent labels     → best for imbalanced datasets where frequent labels are more important
    - 'macro'   : treats all labels equally                → best when you care about rare labels as much as frequent ones
    - 'weighted': like macro but considers label frequency → best if you want a compromise between macro and micro

    - 'macro' or 'weighted' AUC is often best because AUC isn't as affected by class imbalance as F1/Precision/Recall
    - 'macro'      AUC: usually the best because it treats all labels equally, avoiding the dominance of frequent labels
    - 'weighted'   AUC: similar to macro but considers label frequency
    - 'macro'   PR AUC: best for imbalanced datasets because it treats rare labels fairly
    - 'weighted PR AUC: also good, but slightly biased toward frequent labels

    PR AUC is better than ROC AUC when you care about positive examples in imbalanced data
    """
    average = 'macro'
    preds   = np.zeros_like(probs)

    # Apply per-label tuned threshold
    for label_idx in id2label.keys():
        preds[:, label_idx] = (probs[:, label_idx] > thresholds[label_idx]).astype(int)

    # Compute metrics
    f1        = f1_score       (label_ids, preds, average=average)
    precision = precision_score(label_ids, preds, average=average)
    recall    = recall_score   (label_ids, preds, average=average)
    accuracy  = accuracy_score (label_ids, preds)

    # Compute AUC scores with error handling
    try:
        roc_auc              = roc_auc_score          (label_ids, probs, average=average)
    except ValueError:
        roc_auc              = 0.0

    try:
        precision_recall_auc = average_precision_score(label_ids, probs, average=average)
    except ValueError:
        precision_recall_auc = 0.0

    # Compute per-class metrics (average = None)
    per_class_f1        = f1_score       (label_ids, preds, average=None)
    per_class_precision = precision_score(label_ids, preds, average=None)
    per_class_recall    = recall_score   (label_ids, preds, average=None)

    # Generate classification report
    report = classification_report(label_ids, preds, target_names=id2label.values(), zero_division=0)

    # Store metrics
    metrics = {
        'f1'                   : f1,
        'precision'            : precision,
        'recall'               : recall,
        'accuracy'             : accuracy,
        'roc_auc'              : roc_auc,
        'precision_recall_auc' : precision_recall_auc,
        'thresholds'           : thresholds.tolist(),
        'classification_report': report,
        'per_class_f1'         : per_class_f1,
        'per_class_precision'  : per_class_precision,
        'per_class_recall'     : per_class_recall
    }

    return metrics

In [ ]:
%%capture

def predict_with_optimized_thresholds_cuda(trainer, dataset, threshold_tuning=False, thresholds=None, threshold=threshold):
    """
    Predicts using trainer.predict(), with optional threshold tuning on GPU

    Parameters:
    - trainer         : Hugging Face Trainer or CustomTrainer instance
    - dataset         : dataset to predict on
    - id2label        : dictionary mapping label indices (int) to label names (string)
    - threshold_tuning: boolean to enable thresholds tuning per class (if evaluation, True, if prediction, False)
    - thresholds      : if evaluation, custom thresholds, if prediction, tuned thresholds (from evaluation)

    Returns:
    - best_thresholds (if threshold_tuning=True): optimized threshold for each label
    - best_metrics                              : computed with tuned thresholds whether for evaluation or prediction

    - metrics (if threshold_tuning=False): computed with fixed thresholds
    - metrics (if threshold_tuning=True): computed with tuned thresholds
    - predictions: final binary predictions
    - label_ids  : ground true labels from the dataset
    - best_thresholds (if threshold_tuning=True): optimized threshold per class
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Auto-detect GPU

    # Predict
    predictions_output = trainer.predict(dataset)        # <class 'transformers.trainer_utils.PredictionOutput'> len   = 3
    predictions_np     = predictions_output.predictions  # <class 'numpy.ndarray'>                               shape = (2400, 6)
    label_ids_np       = predictions_output.label_ids    # <class 'numpy.ndarray'>                               shape = (2400, 6)
    metrics_dict       = predictions_output.metrics      # <class 'dict'> (= trainer.evaluate results)           len   = 10

    print("==== predictions_output ====")
    print(f"predictions_output.predictions: {type(predictions_np)} shape={predictions_np.shape} \n{predictions_np}")
    print(f"predictions_output.label_ids  : {type(label_ids_np)}   shape={label_ids_np.shape}   \n{label_ids_np}")
    print(f"predictions_output.metrics    : {type(metrics_dict)}   len={len(metrics_dict)}      \n{json.dumps(metrics_dict, indent=4)}")

    # Convert NumPy arrays to PyTorch tensors (torch.from_numpy() keeps the NumPy array's memory layout, while torch.tensor() creates a new copy)
    logits      = torch.tensor(predictions_np, device=device)  # Move to GPU
    true_labels = torch.tensor(label_ids_np, device=device)    # Move to GPU

    print(f"logits     : {type(logits)}      shape={logits.shape}     \n{logits}")
    print(f"true_labels: {type(true_labels)} shape={true_labels.shape}\n{true_labels}")

    # Convert logits to probabilities using PyTorch (on GPU)
    probs = torch.sigmoid(logits)  # <class 'torch.Tensor'>  shape = (1200, 6)

    print(f"probs: {type(probs)} shape={probs.shape}\n{probs}")
    print("============================")
    print()

    num_labels = probs.shape[1]

    if threshold_tuning:
        best_thresholds = torch.full((num_labels,), threshold, device=device, dtype=torch.float32)  # Default to threshold

        # Define candidate thresholds (on GPU)
        threshold_candidates = torch.linspace(0.05, 0.95, 19, device=device)

        for label_idx in range(num_labels):
            best_f1 = 0
            for threshold in threshold_candidates:
                preds = (probs[:, label_idx] > threshold).int()

                # Convert to CPU for sklearn
                precision, recall, f1, _ = precision_recall_fscore_support(
                    true_labels[:, label_idx].cpu().numpy(),
                    preds.cpu().numpy(),
                    average='binary',
                    zero_division=0
                )

                if f1 > best_f1:
                    best_f1                    = f1
                    best_thresholds[label_idx] = threshold  # Store best threshold

        return probs, true_labels, best_thresholds

    # Apply provided thresholds (or default to 0.5)
    if thresholds is None:
        thresholds = torch.full((num_labels,), 0.5, device=device)  # Default to 0.5
    else:
        thresholds = torch.tensor(thresholds, device=device)  # Move thresholds to GPU

    preds = (probs > thresholds).int()

    return preds, true_labels, None